In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [3]:
month_day = [ -1, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31 ]
res = {'날짜': [], '평균기온': [], '최고기온': [], '최저기온': [], '평균운량': [], '일강수량': []}
res['날짜'] = [f'{year}- {month:02d}- {day:02d}' for year in range(2021, 2023) for month in range(1,13) for day in range(1, month_day[month] + 1)]

for year in range(2021, 2023):
    if ((year - 2012) % 4) != 0: res['날짜'].remove(f'{year}- 02- 29')
    if year == 2022:
        for day in range(1, month_day[12] + 1):
            res['날짜'].remove(f'2022- 12- {day:02d}')

In [4]:
for year in range(2021, 2023):
    for month in range(1, 13):
        if year == 2022 and month == 12: continue
        url = f"https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=131&yy={year}&mm={month}&obs=1"
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "lxml")

        res['평균기온'] += re.findall('평균기온:(.*)℃', soup.text)
        res['최고기온'] += re.findall('최고기온:(.*)℃', soup.text)
        res['최저기온'] += re.findall('최저기온:(.*)℃', soup.text)
        res['평균운량'] += re.findall('평균운량:(.*)', soup.text)
        res['일강수량'] += re.findall('일강수량:(.*?)[m| ]', soup.text)

In [5]:
df = pd.DataFrame(res)
df.set_index('날짜', inplace=True)

df.to_csv('../input/[2021-2022]청주일일날씨.csv')